In [2]:
!pip show earthengine-api
!pip install earthengine-api
!pip install rasterio
!pip install google-colab

In [117]:
import ee
import json

ee.Authenticate()
ee.Initialize(project='mse-uhi')

Extracting Image Data (Singapore)

In [ ]:
# Load the Landsat 8 Collection 2 Tier 2 Image Collection
Landsat8_collection = ee.ImageCollection('LANDSAT/LC08/C02/T2_L2')

# Filter by date and location (example date range and bounds for Singapore)
filtered_collection = Landsat8_collection.filterDate('2022-01-01', '2022-12-31').filterBounds(ee.Geometry.Rectangle([103.59, 1.13, 104.07, 1.47]))

# Get the first image from the filtered collection
Landsat8_image = filtered_collection.first()

# If no image is found, handle this condition
if Landsat8_image is None:
    print("No images found for the specified date and region.")
else:
    image_info = Landsat8_image.getInfo()

    # Retrieving band names and other image properties
    band_names = [band['id'] for band in image_info['bands']]
    other_properties = list(image_info['properties'].keys())

    print("Band Names:", band_names)
    print("Other Properties:", other_properties)

    # Select bands
    Landsat8_data = Landsat8_image.select(band_names)

    # Define the region of interest
    singapore = ee.Geometry.Rectangle(103.59, 1.13, 104.07, 1.47)

Band Names: ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10', 'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT']
Other Properties: ['DATA_SOURCE_ELEVATION', 'WRS_TYPE', 'REFLECTANCE_ADD_BAND_1', 'REFLECTANCE_ADD_BAND_2', 'DATUM', 'REFLECTANCE_ADD_BAND_3', 'REFLECTANCE_ADD_BAND_4', 'REFLECTANCE_ADD_BAND_5', 'REFLECTANCE_ADD_BAND_6', 'REFLECTANCE_ADD_BAND_7', 'system:footprint', 'REFLECTIVE_SAMPLES', 'GROUND_CONTROL_POINTS_VERSION', 'SUN_AZIMUTH', 'DATA_SOURCE_TIRS_STRAY_LIGHT_CORRECTION', 'UTM_ZONE', 'DATE_ACQUIRED', 'ELLIPSOID', 'system:time_end', 'DATA_SOURCE_PRESSURE', 'LANDSAT_PRODUCT_ID', 'STATION_ID', 'TEMPERATURE_ADD_BAND_ST_B10', 'DATA_SOURCE_REANALYSIS', 'REFLECTANCE_MULT_BAND_7', 'system:time_start', 'REFLECTANCE_MULT_BAND_6', 'L1_PROCESSING_LEVEL', 'PROCESSING_SOFTWARE_VERSION', 'L1_DATE_PRODUCT_GENERATED', 'ORIENTATION', 'REFLECTANCE_MULT_BAND_1', 'WRS_ROW', 'REFLECTANCE_MU

In [ ]:
task = ee.batch.Export.image.toDrive(
    image=Landsat8_data.toFloat(),
    description='sg_L8_tier2',
    folder='L8_Data',
    scale=30,
    region=singapore,
    fileFormat='GeoTIFF'
) # check out coordinate system, make sure it's consistent to avoid running into inconsistencies in coordinates/NaN values
task.start()

In [ ]:
print(task.status())

{'state': 'COMPLETED', 'description': 'sg_L8_tier2', 'priority': 100, 'creation_timestamp_ms': 1715698468820, 'update_timestamp_ms': 1715698721400, 'start_timestamp_ms': 1715698475865, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/16sjcvaB9JV3tSH_RQ4t2AOGJ22oGDhMf'], 'attempt': 1, 'batch_eecu_usage_seconds': 18.508819580078125, 'id': 'HNGE44AA5H2QSNQXU6EH6S2H', 'name': 'projects/mse-uhi/operations/HNGE44AA5H2QSNQXU6EH6S2H'}


In [ ]:
import rasterio
import pandas as pd
import numpy as np
import os

current_directory = os.getcwd()

file_path = os.path.join(current_directory, 'sg_L8_tier2.tif')

# Open the exported GeoTIFF file.
with rasterio.open(file_path) as src:
    array = src.read()
    nodata_values = src.nodatavals

    # Get the number of bands, image dimensions
    num_bands = src.count
    height, width = src.shape

# Reshape the array so each row corresponds to a pixel and each column to a band
reshaped_array = array.transpose(1, 2, 0).reshape(-1, num_bands)

# Create a DataFrame from the reshaped array with band names
df = pd.DataFrame(reshaped_array, columns=band_names)

# Add pixel coordinates to the DataFrame
df['x'] = np.tile(np.arange(width), height)
df['y'] = np.repeat(np.arange(height), width)

# Replace NoData values in all columns appropriately
for i, nodata_value in enumerate(nodata_values):
    if nodata_value is not None:
        df.iloc[:, i].replace(nodata_value, np.nan, inplace=True)

print(df)

           SR_B1    SR_B2    SR_B3    SR_B4    SR_B5    SR_B6    SR_B7  \
0        29886.0  29660.0  28603.0  28251.0  30884.0  22099.0  18719.0   
1        29205.0  29104.0  28086.0  27782.0  30484.0  21758.0  18476.0   
2        28574.0  28517.0  27575.0  27314.0  30066.0  21411.0  18253.0   
3        28204.0  28047.0  27388.0  27036.0  29794.0  21311.0  18195.0   
4        28167.0  27928.0  27628.0  27139.0  29799.0  21499.0  18258.0   
...          ...      ...      ...      ...      ...      ...      ...   
2236405  50127.0  49691.0  47437.0  46772.0  44830.0  15977.0  18241.0   
2236406  50111.0  49707.0  47451.0  46782.0  44855.0  15936.0  18219.0   
2236407  50114.0  49679.0  47474.0  46775.0  44827.0  15943.0  18198.0   
2236408  50148.0  49676.0  47519.0  46811.0  44842.0  15868.0  18135.0   
2236409  50182.0  49729.0  47587.0  46900.0  44927.0  15795.0  18055.0   

         SR_QA_AEROSOL  ST_B10  ST_ATRAN  ...  ST_DRAD  ST_EMIS  ST_EMSD  \
0                224.0   293.0    3

In [ ]:
# Extracting metadata
metadata_dict = Landsat8_image.getInfo()
metadata_json = json.dumps(metadata_dict)

with open('landsat8_metadata.json', 'w') as f:
    f.write(metadata_json)

metadata_df = pd.DataFrame.from_dict(metadata_dict['properties'], orient='index', columns=['value'])
metadata_df = metadata_df.transpose()

print(metadata_df)

      DATA_SOURCE_ELEVATION WRS_TYPE REFLECTANCE_ADD_BAND_1  \
value               GLS2000        2                   -0.2   

      REFLECTANCE_ADD_BAND_2  DATUM REFLECTANCE_ADD_BAND_3  \
value                   -0.2  WGS84                   -0.2   

      REFLECTANCE_ADD_BAND_4 REFLECTANCE_ADD_BAND_5 REFLECTANCE_ADD_BAND_6  \
value                   -0.2                   -0.2                   -0.2   

      REFLECTANCE_ADD_BAND_7  ... GROUND_CONTROL_POINTS_MODEL SPACECRAFT_ID  \
value                   -0.2  ...                          15     LANDSAT_8   

      TEMPERATURE_MINIMUM_BAND_ST_B10 REFLECTIVE_LINES GEOMETRIC_RMSE_MODEL_Y  \
value                      149.003418             7741                  8.593   

      THERMAL_SAMPLES GEOMETRIC_RMSE_MODEL_X system:asset_size  \
value            7581                 13.291         774239293   

      DATA_SOURCE_AIR_TEMPERATURE          system:index  
value                       MODIS  LC08_125059_20220620  

[1 rows x 92 column

Extracting Image Data (Default)

In [91]:
Landsat8_image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_170052_20170108')

image_info = Landsat8_image.getInfo()

# Retrieving band names and other image properties
band_names = [band['id'] for band in image_info['bands']]
other_properties = list(image_info['properties'].keys())

print(band_names)
print(other_properties)

Landsat8_data = Landsat8_image.select(band_names)

default = ee.Geometry.Rectangle(37.07, 11.50, 37.39, 11.82)

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'BQA']
['RADIANCE_MULT_BAND_5', 'RADIANCE_MULT_BAND_6', 'RADIANCE_MULT_BAND_3', 'RADIANCE_MULT_BAND_4', 'RADIANCE_MULT_BAND_1', 'RADIANCE_MULT_BAND_2', 'K2_CONSTANT_BAND_11', 'K2_CONSTANT_BAND_10', 'system:footprint', 'REFLECTIVE_SAMPLES', 'SUN_AZIMUTH', 'CPF_NAME', 'DATE_ACQUIRED', 'ELLIPSOID', 'google:registration_offset_x', 'google:registration_offset_y', 'STATION_ID', 'RESAMPLING_OPTION', 'ORIENTATION', 'WRS_ROW', 'RADIANCE_MULT_BAND_9', 'TARGET_WRS_ROW', 'RADIANCE_MULT_BAND_7', 'RADIANCE_MULT_BAND_8', 'IMAGE_QUALITY_TIRS', 'TRUNCATION_OLI', 'CLOUD_COVER', 'GEOMETRIC_RMSE_VERIFY', 'COLLECTION_CATEGORY', 'GRID_CELL_SIZE_REFLECTIVE', 'CLOUD_COVER_LAND', 'GEOMETRIC_RMSE_MODEL', 'COLLECTION_NUMBER', 'IMAGE_QUALITY_OLI', 'LANDSAT_SCENE_ID', 'WRS_PATH', 'google:registration_count', 'PANCHROMATIC_SAMPLES', 'PANCHROMATIC_LINES', 'GEOMETRIC_RMSE_MODEL_Y', 'REFLECTIVE_LINES', 'TIRS_STRAY_LIGHT_CORRECTION_SOURCE', 'GEOMETRIC

In [107]:
task = ee.batch.Export.image.toDrive(
    image=Landsat8_data.toFloat(),
    description='default_L8',
    folder='L8_Data',
    scale=30,
    region=default,
    fileFormat='GeoTIFF'
) # check out coordinate system, make sure it's consistent to avoid running into inconsistencies in coordinates/NaN values
task.start()

In [114]:
print(task.status())

{'state': 'COMPLETED', 'description': 'default_L8', 'priority': 100, 'creation_timestamp_ms': 1715681374174, 'update_timestamp_ms': 1715681427634, 'start_timestamp_ms': 1715681377304, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/16sjcvaB9JV3tSH_RQ4t2AOGJ22oGDhMf'], 'attempt': 1, 'batch_eecu_usage_seconds': 9.297932624816895, 'id': 'ZOCVUZVLXI7CDSP2LVFUSQ6C', 'name': 'projects/mse-uhi/operations/ZOCVUZVLXI7CDSP2LVFUSQ6C'}


In [121]:
import rasterio
import pandas as pd
import numpy as np
import os

current_directory = os.getcwd()

file_path = os.path.join(current_directory, 'default_L8.tif')

# Open the exported GeoTIFF file.
with rasterio.open(file_path) as src:
    array = src.read()
    nodata_values = src.nodatavals

    # Get the number of bands, image dimensions
    num_bands = src.count
    height, width = src.shape

# Reshape the array so each row corresponds to a pixel and each column to a band
reshaped_array = array.transpose(1, 2, 0).reshape(-1, num_bands)

# Create a DataFrame from the reshaped array with band names
df = pd.DataFrame(reshaped_array, columns=band_names)

# Add pixel coordinates to the DataFrame
df['x'] = np.tile(np.arange(width), height)
df['y'] = np.repeat(np.arange(height), width)

# Replace NoData values in all columns appropriately
for i, nodata_value in enumerate(nodata_values):
    if nodata_value is not None:
        df.iloc[:, i].replace(nodata_value, np.nan, inplace=True)

print(df)

               B1        B2        B3        B4        B5        B6        B7  \
0        0.120086  0.111088  0.115314  0.148553  0.217867  0.242026  0.204779   
1        0.122186  0.112042  0.114796  0.150462  0.227111  0.253506  0.208596   
2        0.119186  0.108170  0.111524  0.149398  0.219530  0.252170  0.199952   
3        0.118314  0.108143  0.108770  0.143754  0.228720  0.274420  0.202025   
4        0.132220  0.124258  0.134074  0.180347  0.265176  0.302451  0.221766   
...           ...       ...       ...       ...       ...       ...       ...   
1391143  0.104952  0.089219  0.088047  0.072586  0.292389  0.135847  0.072259   
1391144  0.108116  0.095736  0.099853  0.095954  0.269921  0.150026  0.084175   
1391145  0.107707  0.093882  0.094564  0.087774  0.272184  0.146235  0.086138   
1391146  0.112887  0.100372  0.102117  0.102171  0.254460  0.168813  0.107434   
1391147  0.118723  0.108497  0.112260  0.123358  0.234500  0.200743  0.135574   

               B8        B9

In [123]:
# Extracting metadata separately
metadata_dict = Landsat8_image.getInfo()
metadata_json = json.dumps(metadata_dict)

with open('landsat8_metadata.json', 'w') as f:
    f.write(metadata_json)

metadata_df = pd.DataFrame.from_dict(metadata_dict['properties'], orient='index', columns=['value'])
metadata_df = metadata_df.transpose()

print(metadata_df)

      RADIANCE_MULT_BAND_5 RADIANCE_MULT_BAND_6 RADIANCE_MULT_BAND_3  \
value             0.006323             0.001572             0.012253   

      RADIANCE_MULT_BAND_4 RADIANCE_MULT_BAND_1 RADIANCE_MULT_BAND_2  \
value             0.010332             0.012985             0.013296   

      K2_CONSTANT_BAND_11 K2_CONSTANT_BAND_10  \
value         1201.144165         1321.078857   

                                        system:footprint REFLECTIVE_SAMPLES  \
value  {'type': 'LinearRing', 'coordinates': [[35.758...               7631   

       ... SATURATION_BAND_8 SATURATION_BAND_9 TARGET_WRS_PATH  \
value  ...                 N                 N             170   

      RADIANCE_MULT_BAND_11 RADIANCE_MULT_BAND_10 GROUND_CONTROL_POINTS_MODEL  \
value              0.000334              0.000334                         379   

      SPACECRAFT_ID ELEVATION_SOURCE THERMAL_SAMPLES  \
value     LANDSAT_8          GLS2000            7631   

      GROUND_CONTROL_POINTS_VERIFY  
value 